<a href="https://colab.research.google.com/github/FlamesLLC/GPT3Collabs/blob/main/Visual_Scripting_Art_CODEX_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to Image tool
 

## Features 
* complex requests:
  * image and/or text as main prompts  
 

**Run the cell below after each session restart**

Mark `resume` and upload `.pt` file, if you're resuming from the saved params.

In [ ]:
#@title General setup

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  translator = Translator()
except: pass
!pip install ftfy==5.8

import os
import io
import time
from math import exp
import random
import imageio
import numpy as np
import PIL
from skimage import exposure
from base64 import b64encode

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip
model_vit, _ = clip.load('ViT-B/32')

!pip install lpips
import lpips

!git clone https://github.com/eps696/aphantasia
%cd /content/aphantasia/
from clip_fft import to_valid_rgb, fft_image, slice_imgs, checkout
from utils import pad_up_to, basename, img_list, img_read
from progress_bar import ProgressIPy as ProgressBar

workdir = '_out'
tempdir = os.path.join(workdir, 'ttt')
os.makedirs(tempdir, exist_ok=True)

clear_output()

resume = False #@param {type:"boolean"}
if resume:
  resumed = files.upload()
  params_pt = list(resumed.values())[0]

def makevid(seq_dir, size=None):
  out_sequence = seq_dir + '/%03d.jpg'
  out_video = seq_dir + '.mp4'
  !ffmpeg -y -v warning -i $out_sequence $out_video
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

!nvidia-smi -L
print('\nDone!')

Type some `text` and/or upload some image to start.  
`fine_details` input would make micro details follow that topic.  
Put to `subtract` the topics, which you would like to avoid in the result.  
*NB: more prompts = more memory! (handled by auto-decreasing `samples` amount, hopefully you don't need to act).*  
`invert` the whole criteria, if you want to see "the totally opposite".

In [ ]:
#@title Input

text = "" #@param {type:"string"}
fine_details = "" #@param {type:"string"}
subtract = "" #@param {type:"string"}
translate = False #@param {type:"boolean"}
invert = False #@param {type:"boolean"}
upload_image = False #@param {type:"boolean"}

if translate:
  text = translator.translate(text, dest='en').text
if upload_image:
  uploaded = files.upload()

`align` option is about composition (in fact, sampling distribution).  
`sync` value adds SSIM loss between the output and input image (if there's one), allowing to "redraw" it with controlled similarity. 

Turn on `dual_model` to optimize with both CLIP models at once (eats more RAM!).  
Decrease `samples` if you face OOM for higher resolutions (especially when several prompts are used with dual model).  
Setting `steps` much higher (1000-..) will elaborate details and smoother tones, but may start throwing texts like graffiti.

In [ ]:
#@title Generate

# from google.colab import drive
# drive.mount('/content/GDrive')
# clipsDir = '/content/GDrive/MyDrive/T2I ' + dtNow.strftime("%Y-%m-%d %H%M")

!rm -rf tempdir

sideX = 1280 #@param {type:"integer"}
sideY = 720 #@param {type:"integer"}
#@markdown > Tweaks & tuning
align = 'uniform' #@param ['central', 'uniform', 'overscan']
dual_model = False #@param {type:"boolean"}
sync =  0.01 #@param {type:"number"}
#@markdown > Training
steps = 200 #@param {type:"integer"}
samples = 200 #@param {type:"integer"}
learning_rate = .05 
save_freq = 1 #@param {type:"integer"}

if dual_model is True:
  print(' using dual-model optimization')
  model_rn, _ = clip.load('RN50')
  samples = samples // 2
if len(fine_details) > 0:
  samples = int(samples * 0.9)
if len(subtract) > 0:
  samples = int(samples * 0.9)
print(' using %d samples' % samples)

norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
sign = 1. if invert is True else -1.

if upload_image:
  in_img = list(uploaded.values())[0]
  print(' image:', list(uploaded)[0])
  img_in = torch.from_numpy(imageio.imread(in_img).astype(np.float32)/255.).unsqueeze(0).permute(0,3,1,2).cuda()
  in_sliced = slice_imgs([img_in], samples, 224, norm_in)[0]
  img_enc = model_vit.encode_image(in_sliced).detach().clone()
  if dual_model is True:
    img_enc = torch.cat((img_enc, model_rn.encode_image(in_sliced).detach().clone()), 1)
  if sync > 0:
    sim_loss = lpips.LPIPS(net='vgg', verbose=False).cuda()
    img_in = F.interpolate(img_in, (sideY, sideX)).float()
  else:
    del img_in
  del in_sliced; torch.cuda.empty_cache()

if len(text) > 2:
  print(' macro:', text)
  if translate:
    translator = Translator()
    text = translator.translate(text, dest='en').text
    print(' translated to:', text) 
  tx = clip.tokenize(text)
  txt_enc = model_vit.encode_text(tx.cuda()).detach().clone()
  if dual_model is True:
    txt_enc = torch.cat((txt_enc, model_rn.encode_text(tx.cuda()).detach().clone()), 1)

if len(fine_details) > 0:
  print(' micro:', fine_details)
  if translate:
      translator = Translator()
      fine_details = translator.translate(fine_details, dest='en').text
      print(' translated to:', fine_details) 
  tx2 = clip.tokenize(fine_details)
  txt_enc2 = model_vit.encode_text(tx2.cuda()).detach().clone()
  if dual_model is True:
      txt_enc2 = torch.cat((txt_enc2, model_rn.encode_text(tx2.cuda()).detach().clone()), 1)

if len(subtract) > 0:
  print(' without:', subtract)
  if translate:
      translator = Translator()
      subtract = translator.translate(subtract, dest='en').text
      print(' translated to:', subtract) 
  tx0 = clip.tokenize(subtract)
  txt_enc0 = model_vit.encode_text(tx0.cuda()).detach().clone()
  if dual_model is True:
      txt_enc0 = torch.cat((txt_enc0, model_rn.encode_text(tx0.cuda()).detach().clone()), 1)

shape = [1, 3, sideY, sideX]
param_f = fft_image 
# param_f = pixel_image
# learning_rate = 1.
params, image_f = param_f(shape)
image_f = to_valid_rgb(image_f)
optimizer = torch.optim.Adam(params, learning_rate)

def save_img(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def checkout(num):
  with torch.no_grad():
    img = image_f().cpu().numpy()[0]
    if sync > 0 and upload_image:
      img = img **1.5 # empirical tone mapping
  save_img(img, os.path.join(tempdir, '%03d.jpg' % num))
  outpic.clear_output()
  with outpic:
    display(Image('result.jpg'))

def train(i):
  loss = 0
  img_out = image_f()

  micro = False if len(fine_details) > 0 else None
  imgs_sliced = slice_imgs([img_out], samples, 224, norm_in, align, micro=micro)
  out_enc = model_vit.encode_image(imgs_sliced[-1])
  if dual_model is True: # use both clip models
      out_enc = torch.cat((out_enc, model_rn.encode_image(imgs_sliced[-1])), 1)
  if upload_image:
      loss += sign * torch.cosine_similarity(img_enc, out_enc, dim=-1).mean()
  if len(text) > 0: # input text
      loss += sign * torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
  if len(subtract) > 0: # subtract text
      loss += -sign * torch.cosine_similarity(txt_enc0, out_enc, dim=-1).mean()
  if sync > 0 and upload_image: # image composition sync
      prog_sync = (steps - i) / steps 
      loss += prog_sync * sync * sim_loss(F.interpolate(img_out, sim_size).float(), img_in, normalize=True).squeeze()
  if len(fine_details) > 0: # input text for micro details
      imgs_sliced = slice_imgs([img_out], samples, norm_in, align, micro=True)
      out_enc2 = model_vit.encode_image(imgs_sliced[-1])
      if dual_model is True:
          out_enc2 = torch.cat((out_enc2, model_rn.encode_image(imgs_sliced[-1])), 1)
      loss += sign * torch.cosine_similarity(txt_enc2, out_enc2, dim=-1).mean()
      del out_enc2; torch.cuda.empty_cache()
  del img_out, imgs_sliced, out_enc; torch.cuda.empty_cache()

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  if i % save_freq == 0:
    checkout(i // save_freq)

outpic = ipy.Output()
outpic

pbar = ProgressBar(steps)
for i in range(steps):
  train(i)
  _ = pbar.upd()

HTML(makevid(tempdir))
torch.save(params, tempdir + '.pt')
files.download(tempdir + '.pt')
files.download(tempdir + '.mp4')
